References:

https://docs.azuredatabricks.net/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html<br>
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame<br>
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.coalesce<br>
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.repartition<br>

Coalesce: Returns a new DataFrame that has exactly numPartitions partitions.<br>

Repartition: Returns a new DataFrame partitioned by the given partitioning expressions. The resulting DataFrame is hash partitioned.<br>

In [2]:
%run ./adb_1_functions

In [3]:
%run ./adb_3_ingest_to_df

### Partitions and repartitioning

In [5]:
num_partitions = 16

In [6]:
print(df_flights_full.rdd.getNumPartitions())

In [7]:
df2 = df_flights_full.repartition(num_partitions, ["OriginAirportID","DestAirportID"])

In [8]:
# How many partitions were created due to our partitioning expression

print(df2.rdd.getNumPartitions())

### Parquet

In [10]:
parquet_path = "/mnt/hack/parquet/sample/dat202/"

In [11]:
path_coalesce = parquet_path + "coalesce/"
dbutils.fs.rm(path_coalesce, True)

In [12]:
# coalesce(numPartitions: Int): DataFrame - Returns a new DataFrame that has exactly numPartitions partitions

df_flights_full\
  .coalesce(num_partitions)\
  .write\
  .parquet(path_coalesce)

In [13]:
CleanupSparkJobFiles(path_coalesce)

In [14]:
path_repartition = parquet_path + "repartition/"
dbutils.fs.rm(path_repartition, True)

In [15]:
# Repartition and write

df_flights_full\
  .repartition(num_partitions, ["OriginAirportID","DestAirportID"])\
  .write\
  .parquet(path_repartition)

In [16]:
CleanupSparkJobFiles(path_repartition)

In [17]:
path_repartition_partitionby = parquet_path + "repartition-partitionby/"
dbutils.fs.rm(path_repartition_partitionby, True)

In [18]:
# Repartition and write. Here, we are partitioning the output (write.partitionBy) which will create a folder per value in the partition field
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.repartition

df_flights_full\
  .repartition(num_partitions, "Carrier")\
  .write\
  .partitionBy("Carrier")\
  .parquet(path_repartition_partitionby)

In [19]:
CleanupSparkJobFiles(path_repartition_partitionby)